<a href="https://colab.research.google.com/github/ERTUK03/Cats_vs_dogs/blob/main/cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Check available datasets

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

# Load, resize and batch cats_vs_dogs dataset

In [3]:
ds = tfds.load('cats_vs_dogs', split='train', shuffle_files=True, as_supervised=True)
ds = ds.map(lambda image, label: (tf.image.resize(image,(224,224)),label))
ds = ds.batch(32)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete84J7MI/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


# Create our feature extractor for transfer learning

In [4]:
feature_extractor = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    alpha=1.0,
    input_shape=(224,224,3)
)

9406464/9406464 [==============================] - 1s 0us/step


In [5]:
print(feature_extractor.input_shape)

(None, 224, 224, 3)


# Split dataset into train and test

In [6]:
dataset_size=len(ds)
train_size=int(0.9*dataset_size)
test_size=dataset_size-train_size

train_dataset = ds.skip(test_size)
test_dataset = ds.take(test_size)

In [8]:
len(train_dataset)

654

In [9]:
len(test_dataset)

73

# Freeze feature extractor for now

In [10]:
feature_extractor.trainable=False

In [11]:
feature_extractor.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

# Build a basic model with our feature extractor

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./127.5, offset=-1),
    feature_extractor,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(1),
])

In [13]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.fit(train_dataset, epochs=4, validation_data=test_dataset)

Epoch 1/4
654/654 [==============================] - 79s 99ms/step - loss: 0.0859 - accuracy: 0.9683 - val_loss: 0.0417 - val_accuracy: 0.9859
Epoch 2/4
654/654 [==============================] - 57s 82ms/step - loss: 0.0377 - accuracy: 0.9880 - val_loss: 0.0391 - val_accuracy: 0.9859
Epoch 3/4
654/654 [==============================] - 61s 85ms/step - loss: 0.0326 - accuracy: 0.9889 - val_loss: 0.0377 - val_accuracy: 0.9867
Epoch 4/4
654/654 [==============================] - 56s 79ms/step - loss: 0.0305 - accuracy: 0.9897 - val_loss: 0.0356 - val_accuracy: 0.9884


# Unfreeze some layers of feature extractor for fine-tuning

In [15]:
feature_extractor.trainable = True

In [16]:
print("Number of layers in the base model: ", len(feature_extractor.layers))

fine_tune_at = 100

for layer in feature_extractor.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


# Recompile and fine-tune our model

In [17]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [18]:
fine_tune_epochs = 5

model.fit(train_dataset, epochs=fine_tune_epochs, validation_data=test_dataset)

Epoch 1/5
654/654 [==============================] - 84s 99ms/step - loss: 0.0907 - accuracy: 0.9656 - val_loss: 0.0388 - val_accuracy: 0.9906
Epoch 2/5
654/654 [==============================] - 60s 86ms/step - loss: 0.0457 - accuracy: 0.9821 - val_loss: 0.0349 - val_accuracy: 0.9906
Epoch 3/5
654/654 [==============================] - 52s 75ms/step - loss: 0.0310 - accuracy: 0.9880 - val_loss: 0.0352 - val_accuracy: 0.9902
Epoch 4/5
654/654 [==============================] - 52s 73ms/step - loss: 0.0220 - accuracy: 0.9920 - val_loss: 0.0367 - val_accuracy: 0.9889
Epoch 5/5
654/654 [==============================] - 52s 75ms/step - loss: 0.0121 - accuracy: 0.9957 - val_loss: 0.0398 - val_accuracy: 0.9876


# Save the model

In [19]:
model.save('model.keras')

# Create a TFLite model from the previous one with post-training quantization and save

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('tflite_model.tflite', 'wb') as f:
  f.write(tflite_model)